In [1]:
    #from sklearn.model_selection import train_test_split
#from sklearn.model_selection import ParameterGrid

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import IsolationForest

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree

#from sklearn_pandas import DataFrameMapper
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
#from sklearn.metrics import PredictionErrorDisplay

from sklearn.decomposition import PCA

#from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import datetime
import seaborn as sns
sns.set_palette('muted')
sns.set_color_codes('muted')
sns.set_style('white')

import warnings
warnings.filterwarnings('ignore')

from statsmodels.graphics.mosaicplot import mosaic
pd.set_option('display.max_columns', 70)

**downloard the data**

In [2]:
df_geo = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_geo_features_spb.csv')
df_time = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_time_based_features_full.csv')
df_hex_unique = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_hex_unique_with_shares_spb.csv')
df_hex_total = pd.read_csv('C:/Users/elysi/Tele2_project/data/pre_model_state/df_hex_total_with_shares_spb.csv')

In [3]:
df_geo.shape, df_time.shape, df_hex_unique.shape, df_hex_total.shape

((3905, 43), (3892, 63), (3892, 68), (3892, 68))

**Drop part of hexagons with total visits lees or equal 150**

In [31]:
avaible_hexes = df_hex_total[df_hex_total['total_visits'] > 150].h3_9.to_list()
len(avaible_hexes)

1310

In [15]:
df_hex_total = df_hex_total[df_hex_total['h3_9'].isin(avaible_hexes)]
df_hex_unique = df_hex_unique[df_hex_unique['h3_9'].isin(avaible_hexes)]
df_time = df_time[df_time['h3_9'].isin(avaible_hexes)]
df_geo = df_geo[df_geo['h3_9'].isin(avaible_hexes)]

df_geo.shape, df_time.shape, df_hex_unique.shape, df_hex_total.shape

((1310, 43), (1310, 63), (1310, 68), (1310, 68))

In [16]:
df_time.evening_share.mean()

0.21250800932655273

In [17]:
df_time.night_share.mean()

0.2262316274135638

In [18]:
df_time.columns

Index(['h3_9', 'avg_duration_hrs', 'median_duration_hrs', 'weekday_visits',
       'weekend_visits', 'Stay0_05h', 'Stay05_1h', 'Stay1_3h', 'Stay3_6h',
       'Stay6_9h', 'Stay9_15h', 'Stay15_20h', 'Stay20_plus', 'Friday',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday',
       'day', 'evening', 'morning', 'night', 'weekday_visits_share',
       'weekend_visits_share', 'Stay0_05h_share', 'Stay05_1h_share',
       'Stay1_3h_share', 'Stay3_6h_share', 'Stay6_9h_share', 'Stay9_15h_share',
       'Stay15_20h_share', 'Stay20_plus_share', 'Friday_share', 'Monday_share',
       'Saturday_share', 'Sunday_share', 'Thursday_share', 'Tuesday_share',
       'Wednesday_share', 'day_share', 'evening_share', 'morning_share',
       'night_share', 'avg_duration_hrs_per_unique',
       'median_duration_hrs_per_unique', 'Stay0_05h_per_unique',
       'Stay05_1h_per_unique', 'Stay1_3h_per_unique', 'Stay3_6h_per_unique',
       'Stay6_9h_per_unique', 'Stay9_15h_per_unique', 'Stay15

# Transform dataframes for ARL

## df_hex_total dataset

In [11]:
df_hex_total.head(3)

,h3_9,total_visits,car_owners_total,carsharing_user_total,not_citizen_rf_total,healthy_food_inter_total,public_trnsprt_user_total,pharmacies_inter_total,beauty_salon_inter_total,elite_sports_inter_total,sprtswr_ftwr_inter_total,sprt_good_inter_total,trvl_rus_total,trvl_abrd_total,sprt_fit_inter_total,hunting_fishing_inter_total,fashion_inter_total,coffee_inter_total,med_serv_inter_total,education_inter_total,delivery_inter_total,markets_inter_total,no_children_total,age_20_total,age_20_30_total,age_30_40_total,age_40_60_total,age_60_total,male_total,female_total,salary_1_total,salary_2_total,salary_3_total,salary_4_total,salary_5_total,car_owners_total_share,carsharing_user_total_share,not_citizen_rf_total_share,healthy_food_inter_total_share,public_trnsprt_user_total_share,pharmacies_inter_total_share,beauty_salon_inter_total_share,elite_sports_inter_total_share,sprtswr_ftwr_inter_total_share,sprt_good_inter_total_share,trvl_rus_total_share,trvl_abrd_total_share,sprt_fit_inter_total_share,hunting_fishing_inter_total_share,fashion_inter_total_share,coffee_inter_total_share,med_serv_inter_total_share,education_inter_total_share,delivery_inter_total_share,markets_inter_total_share,no_children_total_share,age_20_total_share,age_20_30_total_share,age_30_40_total_share,age_40_60_total_share,age_60_total_share,male_total_share,female_total_share,salary_1_total_share,salary_2_total_share,salary_3_total_share,salary_4_total_share,salary_5_total_share
2,89110600113ffff,333,175,65,204,4,5,177,8,23,0,39,25,3,10,14,94,51,70,75,96,206,257,1,36,165,110,14,191,142,68,62,65,70,50,0.525526,0.195195,0.612613,0.012012,0.015015,0.531532,0.024024,0.069069,0.000000,0.117117,0.075075,0.009009,0.03003,0.042042,0.282282,0.153153,0.210210,0.225225,0.288288,0.618619,0.771772,0.003067,0.110429,0.506135,0.337423,0.042945,0.573574,0.426426,0.215873,0.196825,0.206349,0.222222,0.158730
7,8911060018fffff,439,235,72,253,5,10,258,16,4,0,117,72,13,5,16,33,53,80,54,133,250,397,3,20,143,213,49,247,192,121,114,88,34,62,0.535308,0.164009,0.576310,0.011390,0.022779,0.587699,0.036446,0.009112,0.000000,0.266515,0.164009,0.029613,0.01139,0.036446,0.075171,0.120729,0.182232,0.123007,0.302961,0.569476,0.904328,0.007009,0.046729,0.334112,0.497664,0.114486,0.562642,0.437358,0.288783,0.272076,0.210024,0.081146,0.147971
20,891106004b7ffff,895,508,267,433,36,67,527,72,12,6,173,522,140,30,17,100,158,206,246,322,523,701,4,109,301,377,77,533,362,155,177,155,155,157,0.567598,0.298324,0.483799,0.040223,0.074860,0.588827,0.080447,0.013408,0.006704,0.193296,0.583240,0.156425,0.03352,0.018994,0.111732,0.176536,0.230168,0.274860,0.359777,0.584358,0.783240,0.004608,0.125576,0.346774,0.434332,0.088710,0.595531,0.404469,0.193992,0.221527,0.193992,0.193992,0.196496


**we need to think more what to do with columns where there is percentiles with equal values. currently I will just make several 1s during one hot encoding such process**

**function for creating table with percentile borders**

In [12]:
def filter_columns_and_calculate_percentiles(df, drop_strings, percentiles_list):
    # Filter columns: drop columns containing any of the drop strings
    filtered_columns = [col for col in df.columns if not any(drop_str in col for drop_str in drop_strings)]
    df_filtered = df[filtered_columns]
    
    percentile_data = []
    
    for col in df_filtered.columns:
        percentiles = np.percentile(df_filtered[col], percentiles_list)
        percentile_data.append([col] + list(percentiles))
    
    # Create column names based on specified percentiles
    column_names = ['column_name'] + [f'{p}_percentile' for p in percentiles_list]
    
    # Create a new dataframe with the percentile data
    percentile_df = pd.DataFrame(percentile_data, columns=column_names)
    
    return percentile_df

In [14]:
hex_total_percentiles = filter_columns_and_calculate_percentiles(df_hex_total, ['share', 'h3_9'], [0, 20, 40, 60, 80, 100])
hex_total_percentiles.head(10)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,total_visits,251.0,403.2,568.0,774.6,1069.4,3439.0
1,car_owners_total,37.0,213.2,291.4,402.6,556.8,1914.0
2,carsharing_user_total,9.0,79.0,117.0,166.0,229.8,1164.0
3,not_citizen_rf_total,61.0,211.0,296.0,416.6,580.8,1895.0
4,healthy_food_inter_total,0.0,3.0,8.0,22.0,46.0,224.0
5,public_trnsprt_user_total,0.0,7.0,20.0,35.0,58.8,182.0
6,pharmacies_inter_total,33.0,236.0,327.0,462.6,631.8,2340.0
7,beauty_salon_inter_total,0.0,7.0,21.0,39.0,65.0,292.0
8,elite_sports_inter_total,0.0,0.0,1.0,4.0,15.0,159.0
9,sprtswr_ftwr_inter_total,0.0,0.0,1.0,3.0,14.0,142.0


In [311]:
hex_total_percentiles.shape

(34, 7)

**function for OHE**

1. For each column in df_hex_total that also appears in hex_total_percentiles['column_name'], it created 5 new columns representing the percentile ranges (0_20, 20_40, 40_60, 60_80, 80_100).
2. It then checked in which percentile range the column value falls and marked that range with a 1, leaving the other ranges with 0s.
3. In cases where adjacent percentile values were equal and the column value equaled that as well, it marked multiple ranges with 1s as specified.

In [7]:
def create_adjusted_percentile_columns_with_tracking(hex_total_percentiles, df_hex_total, main_column):
    new_df = df_hex_total[[main_column]].copy()
    # Initialize a list to track columns that get filled in the "if not filled" condition
    not_filled_columns = []
    
    for _, row in hex_total_percentiles.iterrows():
        column_name = row['column_name']
        if column_name in df_hex_total.columns:
            percentiles = row[1:].values
            percentile_ranges = ['0_20', '20_40', '40_60', '60_80', '80_100']
            for pr in percentile_ranges:
                new_col_name = f'{column_name}_{pr}'
                new_df[new_col_name] = 0

            for i, value in df_hex_total.iterrows():
                col_value = value[column_name]
                filled = False
                for j in range(5):
                    low = percentiles[j]
                    high = percentiles[j+1] if j < 4 else percentiles[j]
                    if low < col_value <= high or (j == 0 and col_value == low):
                        new_df.at[i, f'{column_name}_{percentile_ranges[j]}'] = 1
                        filled = True
                    elif col_value == high and j < 4:
                        new_df.at[i, f'{column_name}_{percentile_ranges[j+1]}'] = 1
                        filled = True
                
                if not filled:
                    # If the value is exactly 0 or 1, mark the special case and add to tracking list
                    if col_value == 0:
                        new_df.at[i, f'{column_name}_0_20'] = 1
                        if f'{column_name}_0_20' not in not_filled_columns:
                            not_filled_columns.append(f'{column_name}_0_20')
                    elif col_value == 1:
                        new_df.at[i, f'{column_name}_80_100'] = 1
                        if f'{column_name}_80_100' not in not_filled_columns:
                            not_filled_columns.append(f'{column_name}_80_100')
    
    return new_df, not_filled_columns

**add prcntl ending to mark such columns**

In [313]:
df_hex_total_percentile_ARL, wrong_columns_check = create_adjusted_percentile_columns_with_tracking(hex_total_percentiles, df_hex_total, "h3_9")
for column in df_hex_total_percentile_ARL.columns:
    if column not in ['h3_9']:
        df_hex_total_percentile_ARL.rename(columns={column: column + '_prcntl'}, inplace=True)

In [314]:
wrong_columns_check

[]

In [315]:
df_hex_total_percentile_ARL.shape

(1177, 171)

In [316]:
df_hex_total_percentile_ARL.head(5)

,h3_9,total_visits_0_20_prcntl,total_visits_20_40_prcntl,total_visits_40_60_prcntl,total_visits_60_80_prcntl,total_visits_80_100_prcntl,car_owners_total_0_20_prcntl,car_owners_total_20_40_prcntl,car_owners_total_40_60_prcntl,car_owners_total_60_80_prcntl,car_owners_total_80_100_prcntl,carsharing_user_total_0_20_prcntl,carsharing_user_total_20_40_prcntl,carsharing_user_total_40_60_prcntl,carsharing_user_total_60_80_prcntl,carsharing_user_total_80_100_prcntl,not_citizen_rf_total_0_20_prcntl,not_citizen_rf_total_20_40_prcntl,not_citizen_rf_total_40_60_prcntl,not_citizen_rf_total_60_80_prcntl,not_citizen_rf_total_80_100_prcntl,healthy_food_inter_total_0_20_prcntl,healthy_food_inter_total_20_40_prcntl,healthy_food_inter_total_40_60_prcntl,healthy_food_inter_total_60_80_prcntl,healthy_food_inter_total_80_100_prcntl,public_trnsprt_user_total_0_20_prcntl,public_trnsprt_user_total_20_40_prcntl,public_trnsprt_user_total_40_60_prcntl,public_trnsprt_user_total_60_80_prcntl,public_trnsprt_user_total_80_100_prcntl,pharmacies_inter_total_0_20_prcntl,pharmacies_inter_total_20_40_prcntl,pharmacies_inter_total_40_60_prcntl,pharmacies_inter_total_60_80_prcntl,...,male_total_0_20_prcntl,male_total_20_40_prcntl,male_total_40_60_prcntl,male_total_60_80_prcntl,male_total_80_100_prcntl,female_total_0_20_prcntl,female_total_20_40_prcntl,female_total_40_60_prcntl,female_total_60_80_prcntl,female_total_80_100_prcntl,salary_1_total_0_20_prcntl,salary_1_total_20_40_prcntl,salary_1_total_40_60_prcntl,salary_1_total_60_80_prcntl,salary_1_total_80_100_prcntl,salary_2_total_0_20_prcntl,salary_2_total_20_40_prcntl,salary_2_total_40_60_prcntl,salary_2_total_60_80_prcntl,salary_2_total_80_100_prcntl,salary_3_total_0_20_prcntl,salary_3_total_20_40_prcntl,salary_3_total_40_60_prcntl,salary_3_total_60_80_prcntl,salary_3_total_80_100_prcntl,salary_4_total_0_20_prcntl,salary_4_total_20_40_prcntl,salary_4_total_40_60_prcntl,salary_4_total_60_80_prcntl,salary_4_total_80_100_prcntl,salary_5_total_0_20_prcntl,salary_5_total_20_40_prcntl,salary_5_total_40_60_prcntl,salary_5_total_60_80_prcntl,salary_5_total_80_100_prcntl
2,89110600113ffff,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
7,8911060018fffff,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0
20,891106004b7ffff,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
21,89110600523ffff,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0
31,891106005d3ffff,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0


**work with share df, for which I decided not to use percentiles, just step in 20 percentage**

In [317]:
df_hex_total.head(5)

,h3_9,total_visits,car_owners_total,carsharing_user_total,not_citizen_rf_total,healthy_food_inter_total,public_trnsprt_user_total,pharmacies_inter_total,beauty_salon_inter_total,elite_sports_inter_total,sprtswr_ftwr_inter_total,sprt_good_inter_total,trvl_rus_total,trvl_abrd_total,sprt_fit_inter_total,hunting_fishing_inter_total,fashion_inter_total,coffee_inter_total,med_serv_inter_total,education_inter_total,delivery_inter_total,markets_inter_total,no_children_total,age_20_total,age_20_30_total,age_30_40_total,age_40_60_total,age_60_total,male_total,female_total,salary_1_total,salary_2_total,salary_3_total,salary_4_total,salary_5_total,car_owners_total_share,carsharing_user_total_share,not_citizen_rf_total_share,healthy_food_inter_total_share,public_trnsprt_user_total_share,pharmacies_inter_total_share,beauty_salon_inter_total_share,elite_sports_inter_total_share,sprtswr_ftwr_inter_total_share,sprt_good_inter_total_share,trvl_rus_total_share,trvl_abrd_total_share,sprt_fit_inter_total_share,hunting_fishing_inter_total_share,fashion_inter_total_share,coffee_inter_total_share,med_serv_inter_total_share,education_inter_total_share,delivery_inter_total_share,markets_inter_total_share,no_children_total_share,age_20_total_share,age_20_30_total_share,age_30_40_total_share,age_40_60_total_share,age_60_total_share,male_total_share,female_total_share,salary_1_total_share,salary_2_total_share,salary_3_total_share,salary_4_total_share,salary_5_total_share
2,89110600113ffff,333,175,65,204,4,5,177,8,23,0,39,25,3,10,14,94,51,70,75,96,206,257,1,36,165,110,14,191,142,68,62,65,70,50,0.525526,0.195195,0.612613,0.012012,0.015015,0.531532,0.024024,0.069069,0.000000,0.117117,0.075075,0.009009,0.030030,0.042042,0.282282,0.153153,0.210210,0.225225,0.288288,0.618619,0.771772,0.003067,0.110429,0.506135,0.337423,0.042945,0.573574,0.426426,0.215873,0.196825,0.206349,0.222222,0.158730
7,8911060018fffff,439,235,72,253,5,10,258,16,4,0,117,72,13,5,16,33,53,80,54,133,250,397,3,20,143,213,49,247,192,121,114,88,34,62,0.535308,0.164009,0.576310,0.011390,0.022779,0.587699,0.036446,0.009112,0.000000,0.266515,0.164009,0.029613,0.011390,0.036446,0.075171,0.120729,0.182232,0.123007,0.302961,0.569476,0.904328,0.007009,0.046729,0.334112,0.497664,0.114486,0.562642,0.437358,0.288783,0.272076,0.210024,0.081146,0.147971
20,891106004b7ffff,895,508,267,433,36,67,527,72,12,6,173,522,140,30,17,100,158,206,246,322,523,701,4,109,301,377,77,533,362,155,177,155,155,157,0.567598,0.298324,0.483799,0.040223,0.074860,0.588827,0.080447,0.013408,0.006704,0.193296,0.583240,0.156425,0.033520,0.018994,0.111732,0.176536,0.230168,0.274860,0.359777,0.584358,0.783240,0.004608,0.125576,0.346774,0.434332,0.088710,0.595531,0.404469,0.193992,0.221527,0.193992,0.193992,0.196496
21,89110600523ffff,499,332,118,291,70,59,283,21,1,17,75,40,2,1,8,30,27,211,112,173,279,472,0,68,98,290,38,280,219,38,116,172,93,58,0.665331,0.236473,0.583166,0.140281,0.118236,0.567134,0.042084,0.002004,0.034068,0.150301,0.080160,0.004008,0.002004,0.016032,0.060120,0.054108,0.422846,0.224449,0.346693,0.559118,0.945892,0.000000,0.137652,0.198381,0.587045,0.076923,0.561122,0.438878,0.079665,0.243187,0.360587,0.194969,0.121593
31,891106005d3ffff,770,484,242,415,38,64,424,45,5,29,41,204,169,16,1,69,148,229,190,202,411,650,8,175,207,282,56,409,361,103,158,175,153,107,0.628571,0.314286,0.538961,0.049351,0.083117,0.550649,0.058442,0.006494,0.037662,0.053247,0.264935,0.219481,0.020779,0.001299,0.089610,0.192208,0.297403,0.246753,0.262338,0.533766,0.844156,0.010989,0.240385,0.284341,0.387363,0.076923,0.531169,0.468831,0.147989,0.227011,0.251437,0.219828,0.153736


1. For each column in df_hex_total_example that contains strings from the provided list (["total", "owners", "user", "citizen"]), it created 5 new columns to represent the value ranges (0.00-0.20, 0.20-0.40, 0.40-0.60, 0.60-0.80, 0.80-1.00).
2. Each of these new columns is named with the original column name plus the range it represents (e.g., total_visits_0_20).
3. The function then checked if the original column's value falls within each range and marked that range with a 1, leaving others with 0s.

In [9]:
def create_binary_range_columns(df, main_column, list_of_strings):
    # Initialize the new dataframe with only the main column
    new_df = df[[main_column]].copy()
    
    # Filter columns based on the list_of_strings
    filtered_columns = [col for col in df.columns if any(s in col for s in list_of_strings)]
    
    # Define the range labels and their corresponding numeric boundaries
    range_labels = ['0_20', '20_40', '40_60', '60_80', '80_100']
    range_boundaries = [(0.00, 0.20), (0.20, 0.40), (0.40, 0.60), (0.60, 0.80), (0.80, 1.00)]
    
    for col in filtered_columns:
        for label, (low, high) in zip(range_labels, range_boundaries):
            # Create a new binary column for the current range
            new_col_name = f'{col}_{label}'
            new_df[new_col_name] = df[col].apply(lambda x: 1 if low < x <= high else 0)
            
            if label == '0_20':
                new_df[new_col_name] = df[col].apply(lambda x: 1 if low <= x <= high else 0)
    
    return new_df

In [319]:
df_hex_total_share_ARl = create_binary_range_columns(df_hex_total, "h3_9", ['share'])
df_hex_total_share_ARl.head(3)

,h3_9,car_owners_total_share_0_20,car_owners_total_share_20_40,car_owners_total_share_40_60,car_owners_total_share_60_80,car_owners_total_share_80_100,carsharing_user_total_share_0_20,carsharing_user_total_share_20_40,carsharing_user_total_share_40_60,carsharing_user_total_share_60_80,carsharing_user_total_share_80_100,not_citizen_rf_total_share_0_20,not_citizen_rf_total_share_20_40,not_citizen_rf_total_share_40_60,not_citizen_rf_total_share_60_80,not_citizen_rf_total_share_80_100,healthy_food_inter_total_share_0_20,healthy_food_inter_total_share_20_40,healthy_food_inter_total_share_40_60,healthy_food_inter_total_share_60_80,healthy_food_inter_total_share_80_100,public_trnsprt_user_total_share_0_20,public_trnsprt_user_total_share_20_40,public_trnsprt_user_total_share_40_60,public_trnsprt_user_total_share_60_80,public_trnsprt_user_total_share_80_100,pharmacies_inter_total_share_0_20,pharmacies_inter_total_share_20_40,pharmacies_inter_total_share_40_60,pharmacies_inter_total_share_60_80,pharmacies_inter_total_share_80_100,beauty_salon_inter_total_share_0_20,beauty_salon_inter_total_share_20_40,beauty_salon_inter_total_share_40_60,beauty_salon_inter_total_share_60_80,...,male_total_share_0_20,male_total_share_20_40,male_total_share_40_60,male_total_share_60_80,male_total_share_80_100,female_total_share_0_20,female_total_share_20_40,female_total_share_40_60,female_total_share_60_80,female_total_share_80_100,salary_1_total_share_0_20,salary_1_total_share_20_40,salary_1_total_share_40_60,salary_1_total_share_60_80,salary_1_total_share_80_100,salary_2_total_share_0_20,salary_2_total_share_20_40,salary_2_total_share_40_60,salary_2_total_share_60_80,salary_2_total_share_80_100,salary_3_total_share_0_20,salary_3_total_share_20_40,salary_3_total_share_40_60,salary_3_total_share_60_80,salary_3_total_share_80_100,salary_4_total_share_0_20,salary_4_total_share_20_40,salary_4_total_share_40_60,salary_4_total_share_60_80,salary_4_total_share_80_100,salary_5_total_share_0_20,salary_5_total_share_20_40,salary_5_total_share_40_60,salary_5_total_share_60_80,salary_5_total_share_80_100
2,89110600113ffff,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0
7,8911060018fffff,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0
20,891106004b7ffff,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0


**making combined df**

In [320]:
df_hex_total_ARL_combined = pd.merge(df_hex_total_percentile_ARL, df_hex_total_share_ARl,
                                how='left', left_on='h3_9', right_on='h3_9')

In [321]:
df_hex_total_ARL_combined.shape

(1177, 336)

In [322]:
df_hex_total_ARL_combined.head(3)

,h3_9,total_visits_0_20_prcntl,total_visits_20_40_prcntl,total_visits_40_60_prcntl,total_visits_60_80_prcntl,total_visits_80_100_prcntl,car_owners_total_0_20_prcntl,car_owners_total_20_40_prcntl,car_owners_total_40_60_prcntl,car_owners_total_60_80_prcntl,car_owners_total_80_100_prcntl,carsharing_user_total_0_20_prcntl,carsharing_user_total_20_40_prcntl,carsharing_user_total_40_60_prcntl,carsharing_user_total_60_80_prcntl,carsharing_user_total_80_100_prcntl,not_citizen_rf_total_0_20_prcntl,not_citizen_rf_total_20_40_prcntl,not_citizen_rf_total_40_60_prcntl,not_citizen_rf_total_60_80_prcntl,not_citizen_rf_total_80_100_prcntl,healthy_food_inter_total_0_20_prcntl,healthy_food_inter_total_20_40_prcntl,healthy_food_inter_total_40_60_prcntl,healthy_food_inter_total_60_80_prcntl,healthy_food_inter_total_80_100_prcntl,public_trnsprt_user_total_0_20_prcntl,public_trnsprt_user_total_20_40_prcntl,public_trnsprt_user_total_40_60_prcntl,public_trnsprt_user_total_60_80_prcntl,public_trnsprt_user_total_80_100_prcntl,pharmacies_inter_total_0_20_prcntl,pharmacies_inter_total_20_40_prcntl,pharmacies_inter_total_40_60_prcntl,pharmacies_inter_total_60_80_prcntl,...,male_total_share_0_20,male_total_share_20_40,male_total_share_40_60,male_total_share_60_80,male_total_share_80_100,female_total_share_0_20,female_total_share_20_40,female_total_share_40_60,female_total_share_60_80,female_total_share_80_100,salary_1_total_share_0_20,salary_1_total_share_20_40,salary_1_total_share_40_60,salary_1_total_share_60_80,salary_1_total_share_80_100,salary_2_total_share_0_20,salary_2_total_share_20_40,salary_2_total_share_40_60,salary_2_total_share_60_80,salary_2_total_share_80_100,salary_3_total_share_0_20,salary_3_total_share_20_40,salary_3_total_share_40_60,salary_3_total_share_60_80,salary_3_total_share_80_100,salary_4_total_share_0_20,salary_4_total_share_20_40,salary_4_total_share_40_60,salary_4_total_share_60_80,salary_4_total_share_80_100,salary_5_total_share_0_20,salary_5_total_share_20_40,salary_5_total_share_40_60,salary_5_total_share_60_80,salary_5_total_share_80_100
0,89110600113ffff,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0
1,8911060018fffff,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0
2,891106004b7ffff,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0


## df_hex_unique dataset

In [323]:
df_hex_unique.head(3)

,h3_9,unique_visits,car_owners_unique,carsharing_user_unique,not_citizen_rf_unique,healthy_food_inter_unique,public_trnsprt_user_unique,pharmacies_inter_unique,beauty_salon_inter_unique,elite_sports_inter_unique,sprtswr_ftwr_inter_unique,sprt_good_inter_unique,trvl_rus_unique,trvl_abrd_unique,sprt_fit_inter_unique,hunting_fishing_inter_unique,fashion_inter_unique,coffee_inter_unique,med_serv_inter_unique,education_inter_unique,delivery_inter_unique,markets_inter_unique,no_children_unique,age_20_unique,age_20_30_unique,age_30_40_unique,age_40_60_unique,age_60_unique,male_unique,female_unique,salary_1_unique,salary_2_unique,salary_3_unique,salary_4_unique,salary_5_unique,car_owners_unique_share,carsharing_user_unique_share,not_citizen_rf_unique_share,healthy_food_inter_unique_share,public_trnsprt_user_unique_share,pharmacies_inter_unique_share,beauty_salon_inter_unique_share,elite_sports_inter_unique_share,sprtswr_ftwr_inter_unique_share,sprt_good_inter_unique_share,trvl_rus_unique_share,trvl_abrd_unique_share,sprt_fit_inter_unique_share,hunting_fishing_inter_unique_share,fashion_inter_unique_share,coffee_inter_unique_share,med_serv_inter_unique_share,education_inter_unique_share,delivery_inter_unique_share,markets_inter_unique_share,no_children_unique_share,age_20_unique_share,age_20_30_unique_share,age_30_40_unique_share,age_40_60_unique_share,age_60_unique_share,male_unique_share,female_unique_share,salary_1_unique_share,salary_2_unique_share,salary_3_unique_share,salary_4_unique_share,salary_5_unique_share
2,89110600113ffff,144,79,31,80,4,5,66,6,1,0,14,21,3,7,4,17,20,32,28,34,66,118,1,10,51,62,13,92,52,42,26,22,20,19,0.548611,0.215278,0.555556,0.027778,0.034722,0.458333,0.041667,0.006944,0.000000,0.097222,0.145833,0.020833,0.048611,0.027778,0.118056,0.138889,0.222222,0.194444,0.236111,0.458333,0.819444,0.007299,0.072993,0.372263,0.452555,0.094891,0.638889,0.361111,0.325581,0.201550,0.170543,0.155039,0.147287
7,8911060018fffff,159,74,22,89,4,8,67,5,3,0,20,34,9,4,2,8,19,28,28,35,66,138,2,12,52,67,18,100,59,44,26,25,27,20,0.465409,0.138365,0.559748,0.025157,0.050314,0.421384,0.031447,0.018868,0.000000,0.125786,0.213836,0.056604,0.025157,0.012579,0.050314,0.119497,0.176101,0.176101,0.220126,0.415094,0.867925,0.013245,0.079470,0.344371,0.443709,0.119205,0.628931,0.371069,0.309859,0.183099,0.176056,0.190141,0.140845
20,891106004b7ffff,526,325,204,264,23,35,324,36,12,6,88,423,97,25,14,68,83,156,151,216,317,390,4,87,172,202,50,280,246,93,88,98,85,123,0.617871,0.387833,0.501901,0.043726,0.066540,0.615970,0.068441,0.022814,0.011407,0.167300,0.804183,0.184411,0.047529,0.026616,0.129278,0.157795,0.296578,0.287072,0.410646,0.602662,0.741445,0.007767,0.168932,0.333981,0.392233,0.097087,0.532319,0.467681,0.190965,0.180698,0.201232,0.174538,0.252567


In [324]:
hex_unique_percentiles = filter_columns_and_calculate_percentiles(df_hex_unique, ['share', 'h3_9'], [0, 20, 40, 60, 80, 100])
hex_unique_percentiles.head(5)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,unique_visits,28.0,77.0,103.4,132.0,177.0,788.0
1,car_owners_unique,11.0,37.0,49.0,64.0,86.0,332.0
2,carsharing_user_unique,3.0,14.0,19.0,25.0,33.0,204.0
3,not_citizen_rf_unique,11.0,40.0,53.0,69.0,91.8,413.0
4,healthy_food_inter_unique,0.0,1.0,3.0,4.0,6.0,30.0


In [325]:
hex_unique_percentiles.shape

(34, 7)

**add prcntl ending to mark such columns**

In [326]:
df_hex_unique_percentile_ARL, wrong_columns_check = create_adjusted_percentile_columns_with_tracking(hex_unique_percentiles, df_hex_unique, "h3_9")
for column in df_hex_unique_percentile_ARL.columns:
    if column not in ['h3_9']:
        df_hex_unique_percentile_ARL.rename(columns={column: column + '_prcntl'}, inplace=True)

In [327]:
wrong_columns_check

[]

In [328]:
df_hex_unique_percentile_ARL.shape

(1177, 171)

In [329]:
df_hex_unique_percentile_ARL.head(5)

,h3_9,unique_visits_0_20_prcntl,unique_visits_20_40_prcntl,unique_visits_40_60_prcntl,unique_visits_60_80_prcntl,unique_visits_80_100_prcntl,car_owners_unique_0_20_prcntl,car_owners_unique_20_40_prcntl,car_owners_unique_40_60_prcntl,car_owners_unique_60_80_prcntl,car_owners_unique_80_100_prcntl,carsharing_user_unique_0_20_prcntl,carsharing_user_unique_20_40_prcntl,carsharing_user_unique_40_60_prcntl,carsharing_user_unique_60_80_prcntl,carsharing_user_unique_80_100_prcntl,not_citizen_rf_unique_0_20_prcntl,not_citizen_rf_unique_20_40_prcntl,not_citizen_rf_unique_40_60_prcntl,not_citizen_rf_unique_60_80_prcntl,not_citizen_rf_unique_80_100_prcntl,healthy_food_inter_unique_0_20_prcntl,healthy_food_inter_unique_20_40_prcntl,healthy_food_inter_unique_40_60_prcntl,healthy_food_inter_unique_60_80_prcntl,healthy_food_inter_unique_80_100_prcntl,public_trnsprt_user_unique_0_20_prcntl,public_trnsprt_user_unique_20_40_prcntl,public_trnsprt_user_unique_40_60_prcntl,public_trnsprt_user_unique_60_80_prcntl,public_trnsprt_user_unique_80_100_prcntl,pharmacies_inter_unique_0_20_prcntl,pharmacies_inter_unique_20_40_prcntl,pharmacies_inter_unique_40_60_prcntl,pharmacies_inter_unique_60_80_prcntl,...,male_unique_0_20_prcntl,male_unique_20_40_prcntl,male_unique_40_60_prcntl,male_unique_60_80_prcntl,male_unique_80_100_prcntl,female_unique_0_20_prcntl,female_unique_20_40_prcntl,female_unique_40_60_prcntl,female_unique_60_80_prcntl,female_unique_80_100_prcntl,salary_1_unique_0_20_prcntl,salary_1_unique_20_40_prcntl,salary_1_unique_40_60_prcntl,salary_1_unique_60_80_prcntl,salary_1_unique_80_100_prcntl,salary_2_unique_0_20_prcntl,salary_2_unique_20_40_prcntl,salary_2_unique_40_60_prcntl,salary_2_unique_60_80_prcntl,salary_2_unique_80_100_prcntl,salary_3_unique_0_20_prcntl,salary_3_unique_20_40_prcntl,salary_3_unique_40_60_prcntl,salary_3_unique_60_80_prcntl,salary_3_unique_80_100_prcntl,salary_4_unique_0_20_prcntl,salary_4_unique_20_40_prcntl,salary_4_unique_40_60_prcntl,salary_4_unique_60_80_prcntl,salary_4_unique_80_100_prcntl,salary_5_unique_0_20_prcntl,salary_5_unique_20_40_prcntl,salary_5_unique_40_60_prcntl,salary_5_unique_60_80_prcntl,salary_5_unique_80_100_prcntl
2,89110600113ffff,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
7,8911060018fffff,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0
20,891106004b7ffff,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21,89110600523ffff,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0
31,891106005d3ffff,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**work with share df, for which I decided not to use percentiles, just step in 20 percentage**

In [330]:
df_hex_unique.head(5)

,h3_9,unique_visits,car_owners_unique,carsharing_user_unique,not_citizen_rf_unique,healthy_food_inter_unique,public_trnsprt_user_unique,pharmacies_inter_unique,beauty_salon_inter_unique,elite_sports_inter_unique,sprtswr_ftwr_inter_unique,sprt_good_inter_unique,trvl_rus_unique,trvl_abrd_unique,sprt_fit_inter_unique,hunting_fishing_inter_unique,fashion_inter_unique,coffee_inter_unique,med_serv_inter_unique,education_inter_unique,delivery_inter_unique,markets_inter_unique,no_children_unique,age_20_unique,age_20_30_unique,age_30_40_unique,age_40_60_unique,age_60_unique,male_unique,female_unique,salary_1_unique,salary_2_unique,salary_3_unique,salary_4_unique,salary_5_unique,car_owners_unique_share,carsharing_user_unique_share,not_citizen_rf_unique_share,healthy_food_inter_unique_share,public_trnsprt_user_unique_share,pharmacies_inter_unique_share,beauty_salon_inter_unique_share,elite_sports_inter_unique_share,sprtswr_ftwr_inter_unique_share,sprt_good_inter_unique_share,trvl_rus_unique_share,trvl_abrd_unique_share,sprt_fit_inter_unique_share,hunting_fishing_inter_unique_share,fashion_inter_unique_share,coffee_inter_unique_share,med_serv_inter_unique_share,education_inter_unique_share,delivery_inter_unique_share,markets_inter_unique_share,no_children_unique_share,age_20_unique_share,age_20_30_unique_share,age_30_40_unique_share,age_40_60_unique_share,age_60_unique_share,male_unique_share,female_unique_share,salary_1_unique_share,salary_2_unique_share,salary_3_unique_share,salary_4_unique_share,salary_5_unique_share
2,89110600113ffff,144,79,31,80,4,5,66,6,1,0,14,21,3,7,4,17,20,32,28,34,66,118,1,10,51,62,13,92,52,42,26,22,20,19,0.548611,0.215278,0.555556,0.027778,0.034722,0.458333,0.041667,0.006944,0.000000,0.097222,0.145833,0.020833,0.048611,0.027778,0.118056,0.138889,0.222222,0.194444,0.236111,0.458333,0.819444,0.007299,0.072993,0.372263,0.452555,0.094891,0.638889,0.361111,0.325581,0.201550,0.170543,0.155039,0.147287
7,8911060018fffff,159,74,22,89,4,8,67,5,3,0,20,34,9,4,2,8,19,28,28,35,66,138,2,12,52,67,18,100,59,44,26,25,27,20,0.465409,0.138365,0.559748,0.025157,0.050314,0.421384,0.031447,0.018868,0.000000,0.125786,0.213836,0.056604,0.025157,0.012579,0.050314,0.119497,0.176101,0.176101,0.220126,0.415094,0.867925,0.013245,0.079470,0.344371,0.443709,0.119205,0.628931,0.371069,0.309859,0.183099,0.176056,0.190141,0.140845
20,891106004b7ffff,526,325,204,264,23,35,324,36,12,6,88,423,97,25,14,68,83,156,151,216,317,390,4,87,172,202,50,280,246,93,88,98,85,123,0.617871,0.387833,0.501901,0.043726,0.066540,0.615970,0.068441,0.022814,0.011407,0.167300,0.804183,0.184411,0.047529,0.026616,0.129278,0.157795,0.296578,0.287072,0.410646,0.602662,0.741445,0.007767,0.168932,0.333981,0.392233,0.097087,0.532319,0.467681,0.190965,0.180698,0.201232,0.174538,0.252567
21,89110600523ffff,90,50,18,48,6,4,40,4,1,1,14,20,2,1,4,4,9,21,19,30,41,79,0,12,28,36,10,52,38,13,13,24,18,14,0.555556,0.200000,0.533333,0.066667,0.044444,0.444444,0.044444,0.011111,0.011111,0.155556,0.222222,0.022222,0.011111,0.044444,0.044444,0.100000,0.233333,0.211111,0.333333,0.455556,0.877778,0.000000,0.139535,0.325581,0.418605,0.116279,0.577778,0.422222,0.158537,0.158537,0.292683,0.219512,0.170732
31,891106005d3ffff,427,233,107,236,11,20,211,19,5,2,33,156,130,8,1,36,43,95,98,114,212,354,5,90,126,138,48,260,167,64,89,81,65,76,0.545667,0.250585,0.552693,0.025761,0.046838,0.494145,0.044496,0.011710,0.004684,0.077283,0.365340,0.304450,0.018735,0.002342,0.084309,0.100703,0.222482,0.229508,0.266979,0.496487,0.829040,0.012285,0.221130,0.309582,0.339066,0.117936,0.608899,0.391101,0.170667,0.237333,0.216000,0.173333,0.202667


1. For each column in df_hex_total_example that contains strings from the provided list (["total", "owners", "user", "citizen"]), it created 5 new columns to represent the value ranges (0.00-0.20, 0.20-0.40, 0.40-0.60, 0.60-0.80, 0.80-1.00).
2. Each of these new columns is named with the original column name plus the range it represents (e.g., total_visits_0_20).
3. The function then checked if the original column's value falls within each range and marked that range with a 1, leaving others with 0s.

In [331]:
df_hex_unique_share_ARl = create_binary_range_columns(df_hex_unique, "h3_9", ['share'])
df_hex_unique_share_ARl.head(3)

,h3_9,car_owners_unique_share_0_20,car_owners_unique_share_20_40,car_owners_unique_share_40_60,car_owners_unique_share_60_80,car_owners_unique_share_80_100,carsharing_user_unique_share_0_20,carsharing_user_unique_share_20_40,carsharing_user_unique_share_40_60,carsharing_user_unique_share_60_80,carsharing_user_unique_share_80_100,not_citizen_rf_unique_share_0_20,not_citizen_rf_unique_share_20_40,not_citizen_rf_unique_share_40_60,not_citizen_rf_unique_share_60_80,not_citizen_rf_unique_share_80_100,healthy_food_inter_unique_share_0_20,healthy_food_inter_unique_share_20_40,healthy_food_inter_unique_share_40_60,healthy_food_inter_unique_share_60_80,healthy_food_inter_unique_share_80_100,public_trnsprt_user_unique_share_0_20,public_trnsprt_user_unique_share_20_40,public_trnsprt_user_unique_share_40_60,public_trnsprt_user_unique_share_60_80,public_trnsprt_user_unique_share_80_100,pharmacies_inter_unique_share_0_20,pharmacies_inter_unique_share_20_40,pharmacies_inter_unique_share_40_60,pharmacies_inter_unique_share_60_80,pharmacies_inter_unique_share_80_100,beauty_salon_inter_unique_share_0_20,beauty_salon_inter_unique_share_20_40,beauty_salon_inter_unique_share_40_60,beauty_salon_inter_unique_share_60_80,...,male_unique_share_0_20,male_unique_share_20_40,male_unique_share_40_60,male_unique_share_60_80,male_unique_share_80_100,female_unique_share_0_20,female_unique_share_20_40,female_unique_share_40_60,female_unique_share_60_80,female_unique_share_80_100,salary_1_unique_share_0_20,salary_1_unique_share_20_40,salary_1_unique_share_40_60,salary_1_unique_share_60_80,salary_1_unique_share_80_100,salary_2_unique_share_0_20,salary_2_unique_share_20_40,salary_2_unique_share_40_60,salary_2_unique_share_60_80,salary_2_unique_share_80_100,salary_3_unique_share_0_20,salary_3_unique_share_20_40,salary_3_unique_share_40_60,salary_3_unique_share_60_80,salary_3_unique_share_80_100,salary_4_unique_share_0_20,salary_4_unique_share_20_40,salary_4_unique_share_40_60,salary_4_unique_share_60_80,salary_4_unique_share_80_100,salary_5_unique_share_0_20,salary_5_unique_share_20_40,salary_5_unique_share_40_60,salary_5_unique_share_60_80,salary_5_unique_share_80_100
2,89110600113ffff,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
7,8911060018fffff,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
20,891106004b7ffff,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0


**making combined df**

In [332]:
df_hex_unique_ARL_combined = pd.merge(df_hex_unique_percentile_ARL, df_hex_unique_share_ARl,
                                how='left', left_on='h3_9', right_on='h3_9')

In [333]:
df_hex_unique_ARL_combined.shape

(1177, 336)

In [334]:
df_hex_unique_ARL_combined.head(3)

,h3_9,unique_visits_0_20_prcntl,unique_visits_20_40_prcntl,unique_visits_40_60_prcntl,unique_visits_60_80_prcntl,unique_visits_80_100_prcntl,car_owners_unique_0_20_prcntl,car_owners_unique_20_40_prcntl,car_owners_unique_40_60_prcntl,car_owners_unique_60_80_prcntl,car_owners_unique_80_100_prcntl,carsharing_user_unique_0_20_prcntl,carsharing_user_unique_20_40_prcntl,carsharing_user_unique_40_60_prcntl,carsharing_user_unique_60_80_prcntl,carsharing_user_unique_80_100_prcntl,not_citizen_rf_unique_0_20_prcntl,not_citizen_rf_unique_20_40_prcntl,not_citizen_rf_unique_40_60_prcntl,not_citizen_rf_unique_60_80_prcntl,not_citizen_rf_unique_80_100_prcntl,healthy_food_inter_unique_0_20_prcntl,healthy_food_inter_unique_20_40_prcntl,healthy_food_inter_unique_40_60_prcntl,healthy_food_inter_unique_60_80_prcntl,healthy_food_inter_unique_80_100_prcntl,public_trnsprt_user_unique_0_20_prcntl,public_trnsprt_user_unique_20_40_prcntl,public_trnsprt_user_unique_40_60_prcntl,public_trnsprt_user_unique_60_80_prcntl,public_trnsprt_user_unique_80_100_prcntl,pharmacies_inter_unique_0_20_prcntl,pharmacies_inter_unique_20_40_prcntl,pharmacies_inter_unique_40_60_prcntl,pharmacies_inter_unique_60_80_prcntl,...,male_unique_share_0_20,male_unique_share_20_40,male_unique_share_40_60,male_unique_share_60_80,male_unique_share_80_100,female_unique_share_0_20,female_unique_share_20_40,female_unique_share_40_60,female_unique_share_60_80,female_unique_share_80_100,salary_1_unique_share_0_20,salary_1_unique_share_20_40,salary_1_unique_share_40_60,salary_1_unique_share_60_80,salary_1_unique_share_80_100,salary_2_unique_share_0_20,salary_2_unique_share_20_40,salary_2_unique_share_40_60,salary_2_unique_share_60_80,salary_2_unique_share_80_100,salary_3_unique_share_0_20,salary_3_unique_share_20_40,salary_3_unique_share_40_60,salary_3_unique_share_60_80,salary_3_unique_share_80_100,salary_4_unique_share_0_20,salary_4_unique_share_20_40,salary_4_unique_share_40_60,salary_4_unique_share_60_80,salary_4_unique_share_80_100,salary_5_unique_share_0_20,salary_5_unique_share_20_40,salary_5_unique_share_40_60,salary_5_unique_share_60_80,salary_5_unique_share_80_100
0,89110600113ffff,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,8911060018fffff,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
2,891106004b7ffff,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0


## df_time dataset

In [335]:
df_time.head(3)

,h3_9,avg_duration_hrs,median_duration_hrs,weekday_visits,weekend_visits,Stay0_05h,Stay05_1h,Stay1_3h,Stay3_6h,Stay6_9h,Stay9_15h,Stay15_20h,Stay20_plus,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,day,evening,morning,night,weekday_visits_share,weekend_visits_share,Stay0_05h_share,Stay05_1h_share,Stay1_3h_share,Stay3_6h_share,Stay6_9h_share,Stay9_15h_share,Stay15_20h_share,Stay20_plus_share,Friday_share,Monday_share,Saturday_share,Sunday_share,Thursday_share,Tuesday_share,Wednesday_share,day_share,evening_share,morning_share,night_share,avg_duration_hrs_per_unique,median_duration_hrs_per_unique,Stay0_05h_per_unique,Stay05_1h_per_unique,Stay1_3h_per_unique,Stay3_6h_per_unique,Stay6_9h_per_unique,Stay9_15h_per_unique,Stay15_20h_per_unique,Stay20_plus_per_unique,Stay0_05h_per_unique_share_per_unique,Stay05_1h_per_unique_share_per_unique,Stay1_3h_per_unique_share_per_unique,Stay3_6h_per_unique_share_per_unique,Stay6_9h_per_unique_share_per_unique,Stay9_15h_per_unique_share_per_unique,Stay15_20h_per_unique_share_per_unique,Stay20_plus_per_unique_share_per_unique
2,89110600113ffff,4.214955,1.72,213,120,0,28,229,39,18,9,7,3,46,61,62,58,36,33,37,148,51,84,50,0.639640,0.360360,0.0,0.084084,0.687688,0.117117,0.054054,0.027027,0.021021,0.009009,0.138138,0.183183,0.186186,0.174174,0.108108,0.099099,0.111111,0.444444,0.153153,0.252252,0.150150,5.845619,1.50,0,20,93,14,7,5,3,2,0.0,0.138889,0.645833,0.097222,0.048611,0.034722,0.020833,0.013889
7,8911060018fffff,3.216036,1.85,288,151,0,44,273,62,29,23,4,4,45,90,67,84,48,42,63,198,84,78,79,0.656036,0.343964,0.0,0.100228,0.621868,0.141230,0.066059,0.052392,0.009112,0.009112,0.102506,0.205011,0.152620,0.191344,0.109339,0.095672,0.143508,0.451025,0.191344,0.177677,0.179954,3.199664,1.62,0,20,91,28,8,8,3,1,0.0,0.125786,0.572327,0.176101,0.050314,0.050314,0.018868,0.006289
20,891106004b7ffff,2.762391,1.81,619,276,0,24,635,159,42,28,4,3,153,148,128,148,123,104,91,318,137,185,255,0.691620,0.308380,0.0,0.026816,0.709497,0.177654,0.046927,0.031285,0.004469,0.003352,0.170950,0.165363,0.143017,0.165363,0.137430,0.116201,0.101676,0.355307,0.153073,0.206704,0.284916,2.571171,1.86,0,13,386,99,16,8,1,3,0.0,0.024715,0.733840,0.188213,0.030418,0.015209,0.001901,0.005703


In [336]:
time_percentiles = filter_columns_and_calculate_percentiles(df_time, ['share', 'h3_9'], [0, 20, 40, 60, 80, 100])
time_percentiles.head(5)

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,avg_duration_hrs,2.635169,4.446039,5.143196,5.735293,6.522689,12.019983
1,median_duration_hrs,1.550000,2.090000,2.450000,2.730000,3.050000,7.990000
2,weekday_visits,148.000000,260.000000,365.400000,503.000000,700.000000,2310.000000
3,weekend_visits,68.000000,143.000000,203.000000,272.000000,371.000000,1129.000000
4,Stay0_05h,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [337]:
time_percentiles.shape

(33, 7)

**add prcntl ending to mark such columns**

In [338]:
df_time_percentile_ARL, wrong_columns_check = create_adjusted_percentile_columns_with_tracking(time_percentiles, df_time, "h3_9")
for column in df_time_percentile_ARL.columns:
    if column not in ['h3_9']:
        df_time_percentile_ARL.rename(columns={column: column + '_prcntl'}, inplace=True)

In [339]:
wrong_columns_check

[]

In [340]:
df_time_percentile_ARL.shape

(1177, 166)

In [341]:
df_time_percentile_ARL.head(5)

,h3_9,avg_duration_hrs_0_20_prcntl,avg_duration_hrs_20_40_prcntl,avg_duration_hrs_40_60_prcntl,avg_duration_hrs_60_80_prcntl,avg_duration_hrs_80_100_prcntl,median_duration_hrs_0_20_prcntl,median_duration_hrs_20_40_prcntl,median_duration_hrs_40_60_prcntl,median_duration_hrs_60_80_prcntl,median_duration_hrs_80_100_prcntl,weekday_visits_0_20_prcntl,weekday_visits_20_40_prcntl,weekday_visits_40_60_prcntl,weekday_visits_60_80_prcntl,weekday_visits_80_100_prcntl,weekend_visits_0_20_prcntl,weekend_visits_20_40_prcntl,weekend_visits_40_60_prcntl,weekend_visits_60_80_prcntl,weekend_visits_80_100_prcntl,Stay0_05h_0_20_prcntl,Stay0_05h_20_40_prcntl,Stay0_05h_40_60_prcntl,Stay0_05h_60_80_prcntl,Stay0_05h_80_100_prcntl,Stay05_1h_0_20_prcntl,Stay05_1h_20_40_prcntl,Stay05_1h_40_60_prcntl,Stay05_1h_60_80_prcntl,Stay05_1h_80_100_prcntl,Stay1_3h_0_20_prcntl,Stay1_3h_20_40_prcntl,Stay1_3h_40_60_prcntl,Stay1_3h_60_80_prcntl,...,Stay05_1h_per_unique_0_20_prcntl,Stay05_1h_per_unique_20_40_prcntl,Stay05_1h_per_unique_40_60_prcntl,Stay05_1h_per_unique_60_80_prcntl,Stay05_1h_per_unique_80_100_prcntl,Stay1_3h_per_unique_0_20_prcntl,Stay1_3h_per_unique_20_40_prcntl,Stay1_3h_per_unique_40_60_prcntl,Stay1_3h_per_unique_60_80_prcntl,Stay1_3h_per_unique_80_100_prcntl,Stay3_6h_per_unique_0_20_prcntl,Stay3_6h_per_unique_20_40_prcntl,Stay3_6h_per_unique_40_60_prcntl,Stay3_6h_per_unique_60_80_prcntl,Stay3_6h_per_unique_80_100_prcntl,Stay6_9h_per_unique_0_20_prcntl,Stay6_9h_per_unique_20_40_prcntl,Stay6_9h_per_unique_40_60_prcntl,Stay6_9h_per_unique_60_80_prcntl,Stay6_9h_per_unique_80_100_prcntl,Stay9_15h_per_unique_0_20_prcntl,Stay9_15h_per_unique_20_40_prcntl,Stay9_15h_per_unique_40_60_prcntl,Stay9_15h_per_unique_60_80_prcntl,Stay9_15h_per_unique_80_100_prcntl,Stay15_20h_per_unique_0_20_prcntl,Stay15_20h_per_unique_20_40_prcntl,Stay15_20h_per_unique_40_60_prcntl,Stay15_20h_per_unique_60_80_prcntl,Stay15_20h_per_unique_80_100_prcntl,Stay20_plus_per_unique_0_20_prcntl,Stay20_plus_per_unique_20_40_prcntl,Stay20_plus_per_unique_40_60_prcntl,Stay20_plus_per_unique_60_80_prcntl,Stay20_plus_per_unique_80_100_prcntl
2,89110600113ffff,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,1,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0
7,8911060018fffff,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0
20,891106004b7ffff,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0
21,89110600523ffff,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,1,1,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0
31,891106005d3ffff,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0


**work with share df, for which I decided not to use percentiles, just step in 20 percentage**

In [342]:
df_time.head(5)

,h3_9,avg_duration_hrs,median_duration_hrs,weekday_visits,weekend_visits,Stay0_05h,Stay05_1h,Stay1_3h,Stay3_6h,Stay6_9h,Stay9_15h,Stay15_20h,Stay20_plus,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,day,evening,morning,night,weekday_visits_share,weekend_visits_share,Stay0_05h_share,Stay05_1h_share,Stay1_3h_share,Stay3_6h_share,Stay6_9h_share,Stay9_15h_share,Stay15_20h_share,Stay20_plus_share,Friday_share,Monday_share,Saturday_share,Sunday_share,Thursday_share,Tuesday_share,Wednesday_share,day_share,evening_share,morning_share,night_share,avg_duration_hrs_per_unique,median_duration_hrs_per_unique,Stay0_05h_per_unique,Stay05_1h_per_unique,Stay1_3h_per_unique,Stay3_6h_per_unique,Stay6_9h_per_unique,Stay9_15h_per_unique,Stay15_20h_per_unique,Stay20_plus_per_unique,Stay0_05h_per_unique_share_per_unique,Stay05_1h_per_unique_share_per_unique,Stay1_3h_per_unique_share_per_unique,Stay3_6h_per_unique_share_per_unique,Stay6_9h_per_unique_share_per_unique,Stay9_15h_per_unique_share_per_unique,Stay15_20h_per_unique_share_per_unique,Stay20_plus_per_unique_share_per_unique
2,89110600113ffff,4.214955,1.72,213,120,0,28,229,39,18,9,7,3,46,61,62,58,36,33,37,148,51,84,50,0.639640,0.360360,0.0,0.084084,0.687688,0.117117,0.054054,0.027027,0.021021,0.009009,0.138138,0.183183,0.186186,0.174174,0.108108,0.099099,0.111111,0.444444,0.153153,0.252252,0.150150,5.845619,1.5000,0,20,93,14,7,5,3,2,0.0,0.138889,0.645833,0.097222,0.048611,0.034722,0.020833,0.013889
7,8911060018fffff,3.216036,1.85,288,151,0,44,273,62,29,23,4,4,45,90,67,84,48,42,63,198,84,78,79,0.656036,0.343964,0.0,0.100228,0.621868,0.141230,0.066059,0.052392,0.009112,0.009112,0.102506,0.205011,0.152620,0.191344,0.109339,0.095672,0.143508,0.451025,0.191344,0.177677,0.179954,3.199664,1.6200,0,20,91,28,8,8,3,1,0.0,0.125786,0.572327,0.176101,0.050314,0.050314,0.018868,0.006289
20,891106004b7ffff,2.762391,1.81,619,276,0,24,635,159,42,28,4,3,153,148,128,148,123,104,91,318,137,185,255,0.691620,0.308380,0.0,0.026816,0.709497,0.177654,0.046927,0.031285,0.004469,0.003352,0.170950,0.165363,0.143017,0.165363,0.137430,0.116201,0.101676,0.355307,0.153073,0.206704,0.284916,2.571171,1.8600,0,13,386,99,16,8,1,3,0.0,0.024715,0.733840,0.188213,0.030418,0.015209,0.001901,0.005703
21,89110600523ffff,6.543547,2.80,314,185,0,18,242,71,57,62,27,22,80,79,102,83,37,60,58,194,90,119,96,0.629259,0.370741,0.0,0.036072,0.484970,0.142285,0.114228,0.124248,0.054108,0.044088,0.160321,0.158317,0.204409,0.166333,0.074148,0.120240,0.116232,0.388778,0.180361,0.238477,0.192385,7.963964,1.9075,0,4,53,13,6,10,3,1,0.0,0.044444,0.588889,0.144444,0.066667,0.111111,0.033333,0.011111
31,891106005d3ffff,2.645013,1.58,509,261,0,37,587,80,29,24,10,3,126,112,120,141,80,92,99,293,142,131,204,0.661039,0.338961,0.0,0.048052,0.762338,0.103896,0.037662,0.031169,0.012987,0.003896,0.163636,0.145455,0.155844,0.183117,0.103896,0.119481,0.128571,0.380519,0.184416,0.170130,0.264935,2.345687,1.5500,0,23,340,37,13,11,3,0,0.0,0.053864,0.796253,0.086651,0.030445,0.025761,0.007026,0.000000


In [343]:
df_time_share_ARl = create_binary_range_columns(df_time, "h3_9", ['share'])
df_time_share_ARl.head(3)

,h3_9,weekday_visits_share_0_20,weekday_visits_share_20_40,weekday_visits_share_40_60,weekday_visits_share_60_80,weekday_visits_share_80_100,weekend_visits_share_0_20,weekend_visits_share_20_40,weekend_visits_share_40_60,weekend_visits_share_60_80,weekend_visits_share_80_100,Stay0_05h_share_0_20,Stay0_05h_share_20_40,Stay0_05h_share_40_60,Stay0_05h_share_60_80,Stay0_05h_share_80_100,Stay05_1h_share_0_20,Stay05_1h_share_20_40,Stay05_1h_share_40_60,Stay05_1h_share_60_80,Stay05_1h_share_80_100,Stay1_3h_share_0_20,Stay1_3h_share_20_40,Stay1_3h_share_40_60,Stay1_3h_share_60_80,Stay1_3h_share_80_100,Stay3_6h_share_0_20,Stay3_6h_share_20_40,Stay3_6h_share_40_60,Stay3_6h_share_60_80,Stay3_6h_share_80_100,Stay6_9h_share_0_20,Stay6_9h_share_20_40,Stay6_9h_share_40_60,Stay6_9h_share_60_80,...,Stay05_1h_per_unique_share_per_unique_0_20,Stay05_1h_per_unique_share_per_unique_20_40,Stay05_1h_per_unique_share_per_unique_40_60,Stay05_1h_per_unique_share_per_unique_60_80,Stay05_1h_per_unique_share_per_unique_80_100,Stay1_3h_per_unique_share_per_unique_0_20,Stay1_3h_per_unique_share_per_unique_20_40,Stay1_3h_per_unique_share_per_unique_40_60,Stay1_3h_per_unique_share_per_unique_60_80,Stay1_3h_per_unique_share_per_unique_80_100,Stay3_6h_per_unique_share_per_unique_0_20,Stay3_6h_per_unique_share_per_unique_20_40,Stay3_6h_per_unique_share_per_unique_40_60,Stay3_6h_per_unique_share_per_unique_60_80,Stay3_6h_per_unique_share_per_unique_80_100,Stay6_9h_per_unique_share_per_unique_0_20,Stay6_9h_per_unique_share_per_unique_20_40,Stay6_9h_per_unique_share_per_unique_40_60,Stay6_9h_per_unique_share_per_unique_60_80,Stay6_9h_per_unique_share_per_unique_80_100,Stay9_15h_per_unique_share_per_unique_0_20,Stay9_15h_per_unique_share_per_unique_20_40,Stay9_15h_per_unique_share_per_unique_40_60,Stay9_15h_per_unique_share_per_unique_60_80,Stay9_15h_per_unique_share_per_unique_80_100,Stay15_20h_per_unique_share_per_unique_0_20,Stay15_20h_per_unique_share_per_unique_20_40,Stay15_20h_per_unique_share_per_unique_40_60,Stay15_20h_per_unique_share_per_unique_60_80,Stay15_20h_per_unique_share_per_unique_80_100,Stay20_plus_per_unique_share_per_unique_0_20,Stay20_plus_per_unique_share_per_unique_20_40,Stay20_plus_per_unique_share_per_unique_40_60,Stay20_plus_per_unique_share_per_unique_60_80,Stay20_plus_per_unique_share_per_unique_80_100
2,89110600113ffff,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
7,8911060018fffff,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
20,891106004b7ffff,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0


**making combined df**

In [344]:
df_time_ARL_combined = pd.merge(df_time_percentile_ARL, df_time_share_ARl,
                                how='left', left_on='h3_9', right_on='h3_9')

In [345]:
df_time_ARL_combined.shape

(1177, 311)

In [346]:
df_time_ARL_combined.head(3)

,h3_9,avg_duration_hrs_0_20_prcntl,avg_duration_hrs_20_40_prcntl,avg_duration_hrs_40_60_prcntl,avg_duration_hrs_60_80_prcntl,avg_duration_hrs_80_100_prcntl,median_duration_hrs_0_20_prcntl,median_duration_hrs_20_40_prcntl,median_duration_hrs_40_60_prcntl,median_duration_hrs_60_80_prcntl,median_duration_hrs_80_100_prcntl,weekday_visits_0_20_prcntl,weekday_visits_20_40_prcntl,weekday_visits_40_60_prcntl,weekday_visits_60_80_prcntl,weekday_visits_80_100_prcntl,weekend_visits_0_20_prcntl,weekend_visits_20_40_prcntl,weekend_visits_40_60_prcntl,weekend_visits_60_80_prcntl,weekend_visits_80_100_prcntl,Stay0_05h_0_20_prcntl,Stay0_05h_20_40_prcntl,Stay0_05h_40_60_prcntl,Stay0_05h_60_80_prcntl,Stay0_05h_80_100_prcntl,Stay05_1h_0_20_prcntl,Stay05_1h_20_40_prcntl,Stay05_1h_40_60_prcntl,Stay05_1h_60_80_prcntl,Stay05_1h_80_100_prcntl,Stay1_3h_0_20_prcntl,Stay1_3h_20_40_prcntl,Stay1_3h_40_60_prcntl,Stay1_3h_60_80_prcntl,...,Stay05_1h_per_unique_share_per_unique_0_20,Stay05_1h_per_unique_share_per_unique_20_40,Stay05_1h_per_unique_share_per_unique_40_60,Stay05_1h_per_unique_share_per_unique_60_80,Stay05_1h_per_unique_share_per_unique_80_100,Stay1_3h_per_unique_share_per_unique_0_20,Stay1_3h_per_unique_share_per_unique_20_40,Stay1_3h_per_unique_share_per_unique_40_60,Stay1_3h_per_unique_share_per_unique_60_80,Stay1_3h_per_unique_share_per_unique_80_100,Stay3_6h_per_unique_share_per_unique_0_20,Stay3_6h_per_unique_share_per_unique_20_40,Stay3_6h_per_unique_share_per_unique_40_60,Stay3_6h_per_unique_share_per_unique_60_80,Stay3_6h_per_unique_share_per_unique_80_100,Stay6_9h_per_unique_share_per_unique_0_20,Stay6_9h_per_unique_share_per_unique_20_40,Stay6_9h_per_unique_share_per_unique_40_60,Stay6_9h_per_unique_share_per_unique_60_80,Stay6_9h_per_unique_share_per_unique_80_100,Stay9_15h_per_unique_share_per_unique_0_20,Stay9_15h_per_unique_share_per_unique_20_40,Stay9_15h_per_unique_share_per_unique_40_60,Stay9_15h_per_unique_share_per_unique_60_80,Stay9_15h_per_unique_share_per_unique_80_100,Stay15_20h_per_unique_share_per_unique_0_20,Stay15_20h_per_unique_share_per_unique_20_40,Stay15_20h_per_unique_share_per_unique_40_60,Stay15_20h_per_unique_share_per_unique_60_80,Stay15_20h_per_unique_share_per_unique_80_100,Stay20_plus_per_unique_share_per_unique_0_20,Stay20_plus_per_unique_share_per_unique_20_40,Stay20_plus_per_unique_share_per_unique_40_60,Stay20_plus_per_unique_share_per_unique_60_80,Stay20_plus_per_unique_share_per_unique_80_100
0,89110600113ffff,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,1,1,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,8911060018fffff,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
2,891106004b7ffff,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0


## df_geo dataset

**OHE for distict name**

In [10]:
df_geo = pd.get_dummies(df_geo, columns = ['name'])

In [11]:
adj_col_names = []
for name in df_geo.columns:
    adj_col_names.append(name.replace('name_',''))

df_geo.columns = adj_col_names

In [12]:
df_geo.shape

(1177, 55)

In [13]:
df_geo.head(3)

,h3_9,building_count,highway_count,footway_count,amenity_count,apartments_count,detached_count,dormitory_count,hotel_count,house_count,semidetached_house_count,terrace_count,commercial_count,office_count,retail_count,train_station_count,bus_stop_count,crossing_count,cafe_count,fast_food_count,restaurant_count,college_count,language_school_count,school_count,university_count,atm_count,bank_count,clinic_count,hospital_count,pharmacy_count,theatre_count,townhall_count,bench_count,amount_unqiue_vehicles,unique_vehicle_stops,unique_stops,bus_stop_counter,tram_stop_counter,trolleybus_stop_counter,unique_clubs_names,unique_clubs_sortes,total_fit_clubs,Адмиралтейский район,Василеостровский район,Выборгский район,Калининский район,Кировский район,Красногвардейский район,Красносельский район,Московский район,Невский район,Петроградский район,Приморский район,Фрунзенский район,Центральный район
2,89110600113ffff,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,4.0,0,0,0,0,0,0,0,1,0,0,0,0,0
7,8911060018fffff,1,1,2,9,0,0,0,0,0,0,0,0,0,1,0,1,2,0,2,0,0,0,0,0,5,1,0,0,1,0,0,0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0
20,891106004b7ffff,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,6,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0


**based on the geo_percentiles I am going to remain almost all columns with count in the final df (or I can simply drop it because it looks like dust), so I rename non binary count columns to counter so it will be easy to code. So, now all count columns are binary, counter - no**

In [14]:
df_geo.rename(columns={'amenity_count': 'amenity_counter_osm', 'building_count': 'building_counter_osm'}, inplace=True)

In [16]:
geo_percentiles = filter_columns_and_calculate_percentiles(df_geo, ['район', 'h3_9'], [0, 20, 40, 60, 80, 100])
geo_percentiles

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,building_counter_osm,0.0,2.0,4.0,6.0,9.0,39.0
1,highway_count,0.0,0.0,0.0,1.0,2.0,10.0
2,footway_count,0.0,0.0,1.0,3.0,5.0,22.0
3,amenity_counter_osm,0.0,1.0,3.0,8.0,15.0,95.0
4,apartments_count,0.0,1.0,3.0,5.0,7.0,30.0
5,detached_count,0.0,0.0,0.0,0.0,0.0,19.0
6,dormitory_count,0.0,0.0,0.0,0.0,0.0,6.0
7,hotel_count,0.0,0.0,0.0,0.0,0.0,2.0
8,house_count,0.0,0.0,0.0,0.0,0.0,26.0
9,semidetached_house_count,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
geo_percentiles.shape

(41, 7)

In [400]:
columns_ending_with_count = [col for col in df_geo.columns if col.endswith('_count')]
columns_ending_with_district = [col for col in df_geo.columns if col.endswith('район')]
columns_ending_with_count.extend(columns_ending_with_district)
columns_ending_with_count.append('h3_9')

df_geo_count_dist_osm = df_geo[columns_ending_with_count]


col = df_geo_count_dist_osm.pop('h3_9')
df_geo_count_dist_osm.insert(0, col.name, col)

In [401]:
df_geo_count_dist_osm.head(3)

,h3_9,footway_count,highway_count,crossing_count,bus_stop_count,bench_count,pharmacy_count,apartments_count,bank_count,atm_count,restaurant_count,clinic_count,office_count,school_count,cafe_count,university_count,dormitory_count,retail_count,fast_food_count,commercial_count,college_count,language_school_count,house_count,detached_count,hotel_count,terrace_count,hospital_count,theatre_count,train_station_count,townhall_count,semidetached_house_count,Адмиралтейский район,Василеостровский район,Выборгский район,Калининский район,Кировский район,Колпинский район,Красногвардейский район,Красносельский район,Курортный район,Московский район,Невский район,Петроградский район,Петродворцовый район,Приморский район,Пушкинский район,Фрунзенский район,Центральный район
0,8911060006bffff,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,8911060006bffff,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,89110600077ffff,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [402]:
df_geo_count_dist_osm.shape

(4237, 48)

In [403]:
remove_columns = df_geo_count_dist_osm.columns.to_list()

In [404]:
for column in df_geo_count_dist_osm.columns:
    if column not in ['h3_9']:
        df_geo_count_dist_osm.rename(columns={column: column + '_osm'}, inplace=True)

In [405]:
geo_percentiles_short = filter_columns_and_calculate_percentiles(df_geo, remove_columns, [0, 20, 40, 60, 80, 100])
geo_percentiles_short

,column_name,0_percentile,20_percentile,40_percentile,60_percentile,80_percentile,100_percentile
0,amenity_counter_osm,0.0,0.0,1.0,2.0,4.0,11.0
1,building_counter_osm,0.0,1.0,1.0,1.0,2.0,5.0
2,amount_unqiue_vehicles,0.0,0.0,0.0,1.0,2.0,3.0
3,unique_vehicle_stops,0.0,0.0,0.0,1.0,2.0,9.0
4,unique_stops,0.0,0.0,0.0,1.0,2.0,9.0
5,tram_stop_counter,0.0,0.0,0.0,0.0,0.0,3.0
6,unique_clubs_names,0.0,0.0,0.0,0.0,0.0,6.0
7,unique_clubs_sortes,0.0,0.0,0.0,0.0,0.0,4.0
8,total_fit_clubs,0.0,0.0,0.0,0.0,0.0,6.0


**difficult to do anathink to it, so just lets make it binary, if 0 then 0, if more then 0 then 1**

In [406]:
binary_columns = geo_percentiles_short.column_name.to_list()
binary_columns.append('h3_9')

In [407]:
df_geo_other_columns = df_geo[binary_columns]

col = df_geo_other_columns.pop('h3_9')
df_geo_other_columns.insert(0, col.name, col)
df_geo_other_columns.head(3)

,h3_9,amenity_counter_osm,building_counter_osm,amount_unqiue_vehicles,unique_vehicle_stops,unique_stops,tram_stop_counter,unique_clubs_names,unique_clubs_sortes,total_fit_clubs
0,8911060006bffff,1,0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,8911060006bffff,1,0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,89110600077ffff,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [408]:
binary_columns = geo_percentiles_short.column_name.to_list()
columns_to_replace = binary_columns
df_geo_other_columns[columns_to_replace] = df_geo_other_columns[columns_to_replace].applymap(lambda x: 1 if x > 0 else x)

In [409]:
df_geo_other_columns

,h3_9,amenity_counter_osm,building_counter_osm,amount_unqiue_vehicles,unique_vehicle_stops,unique_stops,tram_stop_counter,unique_clubs_names,unique_clubs_sortes,total_fit_clubs
0,8911060006bffff,1,0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,8911060006bffff,1,0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,89110600077ffff,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,89110600113ffff,1,0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
4,89110600127ffff,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4232,891106b9e4fffff,1,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4233,891106b9e6bffff,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4234,891106b9e6fffff,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4235,891106b9e77ffff,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [410]:
for column in df_geo_other_columns.columns:
    if column not in ['h3_9']:
        df_geo_other_columns.rename(columns={column: column + '_binary'}, inplace=True)

**making combined df**

In [411]:
df_geo_ARL_combined = pd.merge(df_geo_count_dist_osm, df_geo_other_columns,
                                how='left', left_on='h3_9', right_on='h3_9')

In [412]:
df_geo_ARL_combined.shape

(4945, 57)

In [413]:
df_geo_ARL_combined.head(3)

,h3_9,footway_count_osm,highway_count_osm,crossing_count_osm,bus_stop_count_osm,bench_count_osm,pharmacy_count_osm,apartments_count_osm,bank_count_osm,atm_count_osm,restaurant_count_osm,clinic_count_osm,office_count_osm,school_count_osm,cafe_count_osm,university_count_osm,dormitory_count_osm,retail_count_osm,fast_food_count_osm,commercial_count_osm,college_count_osm,language_school_count_osm,house_count_osm,detached_count_osm,hotel_count_osm,terrace_count_osm,hospital_count_osm,theatre_count_osm,train_station_count_osm,townhall_count_osm,semidetached_house_count_osm,Адмиралтейский район_osm,Василеостровский район_osm,Выборгский район_osm,Калининский район_osm,Кировский район_osm,Колпинский район_osm,Красногвардейский район_osm,Красносельский район_osm,Курортный район_osm,Московский район_osm,Невский район_osm,Петроградский район_osm,Петродворцовый район_osm,Приморский район_osm,Пушкинский район_osm,Фрунзенский район_osm,Центральный район_osm,amenity_counter_osm_binary,building_counter_osm_binary,amount_unqiue_vehicles_binary,unique_vehicle_stops_binary,unique_stops_binary,tram_stop_counter_binary,unique_clubs_names_binary,unique_clubs_sortes_binary,total_fit_clubs_binary
0,8911060006bffff,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,8911060006bffff,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,8911060006bffff,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


# Combine all datasets to one for ARL

In [414]:
df_ARL_combined = pd.merge(df_hex_total_ARL_combined, df_hex_unique_ARL_combined,
                                how='left', left_on='h3_9', right_on='h3_9')

df_ARL_combined = pd.merge(df_ARL_combined, df_time_ARL_combined,
                                how='left', left_on='h3_9', right_on='h3_9')

df_ARL_combined = pd.merge(df_ARL_combined, df_geo_ARL_combined,
                                how='left', left_on='h3_9', right_on='h3_9')

In [415]:
df_ARL_combined.shape

(1394, 1037)

In [416]:
df_ARL_combined.head(3)

,h3_9,total_visits_0_20_prcntl,total_visits_20_40_prcntl,total_visits_40_60_prcntl,total_visits_60_80_prcntl,total_visits_80_100_prcntl,car_owners_total_0_20_prcntl,car_owners_total_20_40_prcntl,car_owners_total_40_60_prcntl,car_owners_total_60_80_prcntl,car_owners_total_80_100_prcntl,carsharing_user_total_0_20_prcntl,carsharing_user_total_20_40_prcntl,carsharing_user_total_40_60_prcntl,carsharing_user_total_60_80_prcntl,carsharing_user_total_80_100_prcntl,not_citizen_rf_total_0_20_prcntl,not_citizen_rf_total_20_40_prcntl,not_citizen_rf_total_40_60_prcntl,not_citizen_rf_total_60_80_prcntl,not_citizen_rf_total_80_100_prcntl,healthy_food_inter_total_0_20_prcntl,healthy_food_inter_total_20_40_prcntl,healthy_food_inter_total_40_60_prcntl,healthy_food_inter_total_60_80_prcntl,healthy_food_inter_total_80_100_prcntl,public_trnsprt_user_total_0_20_prcntl,public_trnsprt_user_total_20_40_prcntl,public_trnsprt_user_total_40_60_prcntl,public_trnsprt_user_total_60_80_prcntl,public_trnsprt_user_total_80_100_prcntl,pharmacies_inter_total_0_20_prcntl,pharmacies_inter_total_20_40_prcntl,pharmacies_inter_total_40_60_prcntl,pharmacies_inter_total_60_80_prcntl,...,house_count_osm,detached_count_osm,hotel_count_osm,terrace_count_osm,hospital_count_osm,theatre_count_osm,train_station_count_osm,townhall_count_osm,semidetached_house_count_osm,Адмиралтейский район_osm,Василеостровский район_osm,Выборгский район_osm,Калининский район_osm,Кировский район_osm,Колпинский район_osm,Красногвардейский район_osm,Красносельский район_osm,Курортный район_osm,Московский район_osm,Невский район_osm,Петроградский район_osm,Петродворцовый район_osm,Приморский район_osm,Пушкинский район_osm,Фрунзенский район_osm,Центральный район_osm,amenity_counter_osm_binary,building_counter_osm_binary,amount_unqiue_vehicles_binary,unique_vehicle_stops_binary,unique_stops_binary,tram_stop_counter_binary,unique_clubs_names_binary,unique_clubs_sortes_binary,total_fit_clubs_binary
0,89110600113ffff,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1,8911060018fffff,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,891106004b7ffff,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [417]:
path = 'C:/Users/elysi/Tele2_project/data/pre_model_state/df_ARL_combined_1033_features_31_03_2024.csv'
df_ARL_combined.to_csv(path, index=False)

## Секретные разработки ГРУ (функция делает колонки одновременности для комбинация персентилей)

In [ ]:
def create_percentile_columns(hex_total_percentiles, df_hex_total, main_column):
    new_df = df_hex_total[[main_column]].copy()
    
    for _, row in hex_total_percentiles.iterrows():
        column_name = row['column_name']
        if column_name in df_hex_total.columns:
            percentiles = row[1:].values
            percentile_ranges = ['0_20', '20_40', '40_60', '60_80', '80_100']
            for pr in percentile_ranges:
                new_df[f'{column_name}_{pr}'] = 0

            for i, value in df_hex_total[column_name].iteritems():
                for j in range(5):
                    low, high = percentiles[j], percentiles[j+1] if j < 4 else percentiles[j]
                    if low == high and value == low:
                        new_df.at[i, f'{column_name}_{"_".join(percentile_ranges[j:j+2])}'] = 1
                        if j < 4:  # Ensure not to overflow the index
                            new_df.at[i, f'{column_name}_{"_".join(percentile_ranges[j+1:j+3])}'] = 1
                    elif low <= value < high or (j == 4 and value == high):
                        new_df.at[i, f'{column_name}_{percentile_ranges[j]}'] = 1
                        break
    return new_df

# Checking of columns we drop/save for our ARL model. Since we have 200+ columns which we need to devide to ranges it is better to make some dimension reduction

**Let's check how many columns are binary and check percentage of zeroes in each column**

In [32]:
zroes_geo = pd.DataFrame(df_geo_ohe[df_geo_ohe == 0].count(axis=0)/len(df_geo_ohe.index))
zroes_geo.rename({0: 'zeroes_share'}, axis=1, inplace=True)

In [41]:
zroes_geo.sort_values("zeroes_share", ascending = False).head(10)

,zeroes_share
Курортный район,0.999764
Колпинский район,0.999764
semidetached_house_count,0.999292
Пушкинский район,0.998584
Петродворцовый район,0.998348
townhall_count,0.991975
terrace_count,0.991267
train_station_count,0.985367
hotel_count,0.981355
language_school_count,0.979939
